In [1]:
import pandas as pd
from datetime import datetime, date, time
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.datasets._samples_generator import make_blobs
from sklearn.cluster import KMeans
from fuzzywuzzy import fuzz
#from dateutil import parser
import mysql.connector
from sqlalchemy import create_engine

In [2]:
conn = mysql.connector.connect(host='localhost',username='root',password='0000',database='pro')
my_cursor = conn.cursor()
query = "select * from pro.mapping"
my_cursor.execute(query)
rows = my_cursor.fetchall()
columns = [i[0] for i in my_cursor.description]
dataSQL = pd.DataFrame(rows, columns=columns)
conn.commit()
conn.close()

print("Connection succesfully created!")

Connection succesfully created!


In [3]:
df = pd.read_csv('D:/TT/VNG/test/141123/TEST_OUTPUT_COPY/loyalty_trans_details.csv', encoding='ISO-8859-1', low_memory=False)

df = df[df['event_name']=='Earn']
df = df[df['point_type']!='3']
df = df[df['ref_source'] != 'LP_TRANSFER']
df = df[df['source_type'] != 'EARN_POINT_FROM_AIRDROP_PC_CLIENT']
df = df[df['source_type'] != 'EARN_POINT_FROM_TRANSFER_PROFILE']

df_cus = df.groupby('loyalty_uid')

result = df_cus.agg({
    'loyalty_uid': 'first',
    'log_date': 'max',
    'trans_id': 'count',
    'amount': 'sum' 
})

result.rename(columns={'log_date':'R','trans_id':'F','amount':'M'}, inplace=True)

# result_second = df_cus.agg({
#     'loyalty_uid': 'first',
#     'log_date': 'max',
#     'trans_id': 'count',
#     'amount': 'sum'
# })

# result_second.rename(columns={'log_date':'Recency','trans_id':'Frequency','amount':'Monetary'}, inplace=True)

# result = pd.concat([result.set_index('loyalty_uid'), result_second.set_index('loyalty_uid')], axis=1, join='outer')

# # Reset index để có cột 'loyalty_uid'
#result.reset_index(inplace=True)

#result = result.merge(result_second, on='loyalty_uid')
#print(result)

result['R'] = result['R'].str.extract(r'\[(.*?)\]')  # Extract date strings inside square brackets
result['R'] = pd.to_datetime(result['R'], errors='coerce', format='%Y/%m/%d')


current_date = datetime.now()
result['R'] = (current_date - result['R']).dt.days
result['day count'] = result['R']

In [4]:
df

,id,log_date,block_number,block_time,event_name,trans_id,loyalty_uid,is_ggid,tier_code,conversion_rate,...,total_amount,currency,quantity,game_id,game_client_id,ref_source,source_type,source_time,block_key,updated_time
23,6a8c61db46d4c4ca4aec14ac44497353,[2023/11/22],2706037,[2023/11/22:03:49:25 PM],Earn,2311220849250502516351,3e30fab6d34f7981,GGID,3,0.015,...,89000,VND,1,658.0,10658.0,GPC_PAYMENT,EARN_POINT_FROM_PAYMENT_WEBSHOP,[2023/11/22:03:49:24 PM],c3c4cfe42179d186b678210ad41dae5c,[2023/11/22:04:10:04 PM]
24,be453bb2c98eda506d52cccab6cd0e94,[2023/11/22],2706039,[2023/11/22:03:49:27 PM],Earn,2311220849271684515702,3e007725196ff981,GGID,3,0.015,...,39000,VND,1,662.0,10662.0,GPC_PAYMENT,EARN_POINT_FROM_PAYMENT_WEBSHOP,[2023/11/22:03:49:21 PM],4838404b270f0d0f8c6e82e520a8e230,[2023/11/22:04:10:04 PM]
25,b9d7edb78633b0804a757a82801542cb,[2023/11/22],2706040,[2023/11/22:03:49:29 PM],Earn,2311220849292094016371,3e364ae6650f7981,GGID,3,0.000,...,0,NaN,0,NaN,NaN,LP_QUEST,EARN_POINT_FROM_QUEST_TRACKING,[2023/11/22:03:49:29 PM],417e685a9bdda209ca8f1eefce087f3e,[2023/11/22:04:10:04 PM]
26,3ecfb2ebe15242a1c74e676fa9bbf9fd,[2023/11/22],2706043,[2023/11/22:03:49:31 PM],Earn,2311220849323964015115,3e060bace9b6d981,GGID,3,0.015,...,5000,VND,1,381.0,20381.0,GPC_PAYMENT,EARN_POINT_FROM_PAYMENT_WEBSHOP,[2023/11/22:03:49:17 PM],c10332b04f58a19efd14bd674de39cd8,[2023/11/22:04:10:04 PM]
27,014710015f5b4c992f2f4faebe1bc418,[2023/11/22],2706044,[2023/11/22:03:49:32 PM],Earn,2311220849322534415422,3e421bc1bcf50981,GGID,1,0.010,...,200000,VND,1,381.0,20381.0,GPC_PAYMENT,EARN_POINT_FROM_PAYMENT_WEBSHOP,[2023/11/22:03:49:31 PM],8413289018960a899dc5d09d3e06a07c,[2023/11/22:04:10:04 PM]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3561901,ea30dc6d9994411be2f06faa9885bc46,[2023/11/22],2706077,[2023/11/22:03:50:16 PM],Earn,2311220850161684515710,3e761cc945675981,GGID,-1,1.000,...,0,NaN,0,661.0,NaN,LP_AIRDROP,EARN_POINT_FROM_AIRDROP_EVENT,[2023/11/22:03:50:16 PM],13993dbb6afd629b9d834e5c9ba905c9,[2023/11/22:04:10:17 PM]
3561902,ef19f7637169d13ffd6311f5642014b1,[2023/11/22],2706078,[2023/11/22:03:50:17 PM],Earn,2311220850176466015965,3e06f3d96076d981,GGID,-1,1.000,...,0,NaN,0,122.0,NaN,LP_AIRDROP,EARN_POINT_FROM_AIRDROP_EVENT,[2023/11/22:03:50:17 PM],ada12dac5c21ebe82dd7637d867def40,[2023/11/22:04:10:17 PM]
3561903,b716ca2f0094420f5672ff6e9c2dc1f6,[2023/11/22],2706086,[2023/11/22:03:50:35 PM],Earn,2311220850363964015119,3e059f419bb6d981,GGID,-1,1.000,...,0,NaN,0,661.0,NaN,LP_AIRDROP,EARN_POINT_FROM_AIRDROP_EVENT,[2023/11/22:03:50:36 PM],5e49eaf05fb28bcc11a3e61795e5f2d0,[2023/11/22:04:10:17 PM]
3561904,23ae405b9718a0b45050f562f91bbe48,[2023/11/22],2706103,[2023/11/22:03:50:53 PM],Earn,2311220850543964015122,3e6b9e8272448981,GGID,-1,1.000,...,0,NaN,0,377.0,NaN,LP_AIRDROP,EARN_POINT_FROM_AIRDROP_EVENT,[2023/11/22:03:50:54 PM],7787f4e275156c0070106be485145498,[2023/11/22:04:10:17 PM]


In [5]:
def M_segment(data):
    if (data >= 100000):
        return 5
    elif (data >= 20000):
        return 4
    elif (data >= 10000):
        return 3
    elif (data >= 5000):
        return 2
    else:
        return 1

In [6]:
M = result['M'].values
M_segment_vectorized = np.vectorize(M_segment)
M_transformed = M_segment_vectorized(M)

print(M_transformed)
result['day count'] = M_transformed
result.to_clipboard()

[1 1 1 ... 2 2 5]


In [7]:
R_col = result[['loyalty_uid','R']]
F_col = result[['loyalty_uid','F']]
M_col = result[['loyalty_uid','M']]

R_sorted = R_col.sort_values(by='R', ascending=False)
F_sorted = F_col.sort_values(by='F', ascending=True)
M_sorted = M_col.sort_values(by='M', ascending=True)

# P = [0, 25, 50, 75, 100]
N = len(result)
print(N)

#result_array = np.tile(np.repeat(1, 6), int(np.ceil(N / 5)))[:N]
result_array = np.repeat(np.arange(1, 6), N // 5 + 1)[:N]


R_sorted.loc[:, 'R'] = result_array
F_sorted.loc[:, 'F'] = result_array
M_sorted.loc[:, 'M'] = result_array

# Reset the index of the DataFrames
R_sorted_reset = R_sorted.reset_index(drop=True)
F_sorted_reset = F_sorted.reset_index(drop=True)
M_sorted_reset = M_sorted.reset_index(drop=True)

# Merge the sorted DataFrames on 'loyalty_uid'
merged_result = pd.merge(pd.merge(R_sorted_reset[['loyalty_uid', 'R']], F_sorted_reset[['loyalty_uid', 'F']], on='loyalty_uid'), M_sorted_reset[['loyalty_uid', 'M']], on='loyalty_uid')

# Create a new DataFrame with only the 'loyalty_uid' column
#loyalty_uid_df = merged_result[['loyalty_uid']]



merged_result['RFM Score'] = merged_result['R'].astype(str) + merged_result['F'].astype(str) + merged_result['M'].astype(str)
print(merged_result)
#merged_result = pd.merge(R_sorted[['loyalty_uid', 'R']], F_sorted[['loyalty_uid', 'F']], on='loyalty_uid')
# result = pd.merge(merged_result, M_sorted[['loyalty_uid', 'M']], on='loyalty_uid')
# result
#R_sorted.to_excel("R_sorted.xlsx", index=False)

443894
                             loyalty_uid  R  F  M RFM Score
0       0610_vngjs3__2471824338208522240  1  3  1       131
1               0610_2255884920624816128  1  1  2       112
2               0221_2299448749681844224  1  2  1       121
3               0221_2499811680743915520  1  3  2       132
4               0221_1612771003913601024  1  3  1       131
...                                  ... .. .. ..       ...
443889                  3e7da5c2e1752981  5  5  5       555
443890                  3e76d84768b91981  5  1  2       512
443891                  3e54c87654800981  5  5  5       555
443892                  3e03ed7e81ddf981  5  4  5       545
443893                  3e15c12a0937d981  5  5  5       555

[443894 rows x 5 columns]


In [8]:
new = df_cus.agg({
    'log_date': 'max',
    'trans_id': 'count',
    'amount': 'sum' 
})

new.rename(columns={'log_date':'Recency','trans_id':'Frequency','amount':'Monetary'}, inplace=True)

merged_result_reset = merged_result.reset_index(drop=True)

final_result = pd.merge(merged_result_reset, new, on='loyalty_uid', how='left')

# Display the resulting DataFrame
print(final_result)


                             loyalty_uid  R  F  M RFM Score       Recency  \
0       0610_vngjs3__2471824338208522240  1  3  1       131  [2023/08/14]   
1               0610_2255884920624816128  1  1  2       112  [2023/08/14]   
2               0221_2299448749681844224  1  2  1       121  [2023/08/14]   
3               0221_2499811680743915520  1  3  2       132  [2023/08/14]   
4               0221_1612771003913601024  1  3  1       131  [2023/08/14]   
...                                  ... .. .. ..       ...           ...   
443889                  3e7da5c2e1752981  5  5  5       555  [2023/12/04]   
443890                  3e76d84768b91981  5  1  2       512  [2023/12/04]   
443891                  3e54c87654800981  5  5  5       555  [2023/12/04]   
443892                  3e03ed7e81ddf981  5  4  5       545  [2023/12/04]   
443893                  3e15c12a0937d981  5  5  5       555  [2023/12/04]   

        Frequency  Monetary  
0               2       386  
1              

In [6]:
final_result['Group Customer'] = final_result['RFM Score']

final_result['Group Customer'] = final_result['RFM Score'].map(dataSQL.set_index('id')['name']).fillna('Unknown')
final_result['rank'] = final_result['RFM Score'].map(dataSQL.set_index('id')['stt']).fillna('Unknown')

NameError: name 'final_result' is not defined

In [5]:

final_result.to_excel('test output.xlsx', index=False)
print(final_result)

NameError: name 'final_result' is not defined